# Neural Network Testing with Regular Data

### Get Libraries and Functions

In [1]:
import sys 
sys.path.append("../")

import torch 
import torch.nn

from Conv1d_NN import Conv1d_NN

from models import * 
from train import * 
from dataset import * 



### Regular Data

In [2]:
# Regular Noise Dataset
dataset = MNIST1D_Dataset() 
data = dataset.make_dataset()

### Regular Conv1d_NN

In [3]:
# Regular Conv1d_NN
model1 = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =32, K = 3),  # Change the number of input channels to 1
    nn.ReLU(), 
    Conv1d_NN(in_channels=32, out_channels=32, K = 3),
    nn.ReLU(),
    Conv1d_NN(in_channels=32, out_channels=32, K = 3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1280, 10)
)

# Get summary 
from torchsummary import summary
summary(model1, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 40]             128
         Conv1d_NN-2               [-1, 32, 40]               0
              ReLU-3               [-1, 32, 40]               0
            Conv1d-4               [-1, 32, 40]           3,104
         Conv1d_NN-5               [-1, 32, 40]               0
              ReLU-6               [-1, 32, 40]               0
            Conv1d-7               [-1, 32, 40]           3,104
         Conv1d_NN-8               [-1, 32, 40]               0
              ReLU-9               [-1, 32, 40]               0
          Flatten-10                 [-1, 1280]               0
           Linear-11                   [-1, 10]          12,810
Total params: 19,146
Trainable params: 19,146
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [4]:
# Regular Conv1d_NN Results 
print("Regular Conv1d_NN Results")
results1 = train_model(data, model1, dataset.model_args)

Regular Conv1d_NN Results
step 1000, dt 18.81s, train_loss 9.682e-01, test_loss 1.498e+00, train_acc 65.6, test_acc 45.8
step 2000, dt 17.99s, train_loss 9.179e-01, test_loss 1.688e+00, train_acc 65.3, test_acc 45.4
step 3000, dt 18.93s, train_loss 7.473e-01, test_loss 1.903e+00, train_acc 66.8, test_acc 42.6
step 4000, dt 17.91s, train_loss 7.513e-01, test_loss 1.993e+00, train_acc 67.7, test_acc 45.4
step 5000, dt 18.10s, train_loss 6.896e-01, test_loss 1.922e+00, train_acc 72.0, test_acc 43.3
step 6000, dt 19.01s, train_loss 5.796e-01, test_loss 2.173e+00, train_acc 71.4, test_acc 43.7


### Local -> Global Neural Network

In [5]:
# Local -> Global Conv1d_NN
model2 = nn.Sequential(
   nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(), 
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Flatten(),
   nn.Linear(1088, 10)
)

# Get summary 
from torchsummary import summary 
summary(model2, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 38]             128
              ReLU-2               [-1, 32, 38]               0
            Conv1d-3               [-1, 32, 38]           3,104
         Conv1d_NN-4               [-1, 32, 38]               0
              ReLU-5               [-1, 32, 38]               0
            Conv1d-6               [-1, 32, 36]           3,104
              ReLU-7               [-1, 32, 36]               0
            Conv1d-8               [-1, 32, 36]           3,104
         Conv1d_NN-9               [-1, 32, 36]               0
             ReLU-10               [-1, 32, 36]               0
           Conv1d-11               [-1, 32, 34]           3,104
             ReLU-12               [-1, 32, 34]               0
           Conv1d-13               [-1, 32, 34]           3,104
        Conv1d_NN-14               [-1,

In [6]:
# Local -> Global Conv1d_NN Results 
print("Local -> Global Conv1d_NN Results")
results2 = train_model(data, model2, dataset.model_args)

Local -> Global Conv1d_NN Results
step 1000, dt 19.85s, train_loss 1.000e+00, test_loss 9.344e-01, train_acc 66.8, test_acc 63.6
step 2000, dt 17.94s, train_loss 5.179e-01, test_loss 6.010e-01, train_acc 83.0, test_acc 80.1
step 3000, dt 17.28s, train_loss 5.710e-01, test_loss 6.227e-01, train_acc 82.0, test_acc 78.2
step 4000, dt 17.68s, train_loss 3.069e-01, test_loss 6.436e-01, train_acc 84.0, test_acc 78.2
step 5000, dt 17.39s, train_loss 2.944e-01, test_loss 5.354e-01, train_acc 89.1, test_acc 84.7
step 6000, dt 17.68s, train_loss 1.454e-01, test_loss 6.474e-01, train_acc 88.7, test_acc 84.3


### Global -> Local Neural Network

In [7]:
# Global -> Local Conv1d_NN
model3 = nn.Sequential(
   Conv1d_NN(in_channels=1, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(), 
   nn.Flatten(),
   nn.Linear(1088, 10)
)

# Get summary 
from torchsummary import summary 
summary(model3, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 40]             128
         Conv1d_NN-2               [-1, 32, 40]               0
              ReLU-3               [-1, 32, 40]               0
            Conv1d-4               [-1, 32, 38]           3,104
              ReLU-5               [-1, 32, 38]               0
            Conv1d-6               [-1, 32, 38]           3,104
         Conv1d_NN-7               [-1, 32, 38]               0
              ReLU-8               [-1, 32, 38]               0
            Conv1d-9               [-1, 32, 36]           3,104
             ReLU-10               [-1, 32, 36]               0
           Conv1d-11               [-1, 32, 36]           3,104
        Conv1d_NN-12               [-1, 32, 36]               0
             ReLU-13               [-1, 32, 36]               0
           Conv1d-14               [-1,

In [8]:
# Global -> Local Conv1d_NN Results 
print("Global -> Local Conv1d_NN Results")
results3 = train_model(data, model3, dataset.model_args)

Global -> Local Conv1d_NN Results
step 1000, dt 19.46s, train_loss 4.570e-01, test_loss 5.931e-01, train_acc 85.7, test_acc 81.2
step 2000, dt 20.09s, train_loss 3.671e-01, test_loss 4.458e-01, train_acc 89.4, test_acc 86.4
step 3000, dt 20.24s, train_loss 2.444e-01, test_loss 4.280e-01, train_acc 92.2, test_acc 85.8
step 4000, dt 18.86s, train_loss 3.859e-01, test_loss 5.769e-01, train_acc 91.3, test_acc 83.8
step 5000, dt 20.65s, train_loss 2.847e-01, test_loss 5.142e-01, train_acc 92.2, test_acc 86.0
step 6000, dt 20.44s, train_loss 1.438e-01, test_loss 5.244e-01, train_acc 91.7, test_acc 86.5


### Branching Neural Network

In [9]:
model4 = nn.Sequential(
   BranchingNetwork(in_ch = 1, out_ch1 = 16, out_ch2=16, kernel_size = 3), 
   BranchingNetwork(in_ch = 16, out_ch1 = 8, out_ch2=8, kernel_size = 3),
   BranchingNetwork(in_ch = 8, out_ch1 = 4, out_ch2=4, kernel_size = 3),
   nn.Flatten(), 
   nn.Linear(160, 10)
)


# Get summary
from torchsummary import summary 
summary(model4, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 16, 38]              64
              ReLU-2               [-1, 16, 38]               0
            Conv1d-3               [-1, 16, 40]              64
         Conv1d_NN-4               [-1, 16, 40]               0
              ReLU-5               [-1, 16, 40]               0
            Conv1d-6               [-1, 16, 40]             528
  BranchingNetwork-7               [-1, 16, 40]               0
            Conv1d-8                [-1, 8, 38]             392
              ReLU-9                [-1, 8, 38]               0
           Conv1d-10                [-1, 8, 40]             392
        Conv1d_NN-11                [-1, 8, 40]               0
             ReLU-12                [-1, 8, 40]               0
           Conv1d-13                [-1, 8, 40]             136
 BranchingNetwork-14                [-1

In [10]:
# Branching Network Results 
print("Branching Conv1d_NN Conv1d Results")
results4 = train_model(data, model4, dataset.model_args)

Branching Conv1d_NN Conv1d Results
step 1000, dt 24.98s, train_loss 2.406e-01, test_loss 5.027e-01, train_acc 91.6, test_acc 86.3
step 2000, dt 23.91s, train_loss 3.971e-02, test_loss 4.663e-01, train_acc 96.5, test_acc 89.9
step 3000, dt 23.45s, train_loss 1.728e-01, test_loss 4.960e-01, train_acc 95.4, test_acc 90.2
step 4000, dt 23.68s, train_loss 1.150e-01, test_loss 5.067e-01, train_acc 94.8, test_acc 90.4
step 5000, dt 23.47s, train_loss 2.517e-02, test_loss 4.663e-01, train_acc 97.7, test_acc 91.3
step 6000, dt 23.87s, train_loss 3.281e-02, test_loss 5.402e-01, train_acc 96.5, test_acc 91.1


### 